In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir("..")

In [3]:
from deepsvg.svglib.geom import Point
from deepsvg.svglib.svg import SVG
from deepsvg.svglib.svg_path import SVGPath
from deepsvg.svglib.utils import to_gif

from deepsvg.difflib.tensor import SVGTensor
from deepsvg.difflib.utils import *
from deepsvg.difflib.loss import *

In [4]:
import torch.optim as optim
import IPython.display as ipd
from moviepy.editor import ImageClip, concatenate_videoclips, ipython_display

In [5]:
import pickle
import csv
import pandas as pd 

# Save svg tensor in a .pkl file

In [6]:
def save_svg_tensor(file_path_name, file_obj):
    # save svg tensor data into .pkl file 
    with open(file_path_name, 'wb') as handle:
        pickle.dump(file_obj, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_svg_tensor(file_path_name):
    # load svg tensor data object according to the data path  
    with open(file_path_name, 'rb') as handle:
        svg_tensor_obj = pickle.load(handle)
        return svg_tensor_obj

In [7]:
svg_folder_path = 'dataset/svgs_simplified/'
svg_tensor_folder_path = 'dataset/svgs_tensor/'

In [8]:
svg_file_name_list = os.listdir(svg_folder_path)

In [9]:
# load svg_meta.csv
nonempty_svg_data_list = []
with open('dataset/svgs_meta.csv','rt')as f:
    svg_meta_data = csv.reader(f)
    header = next(svg_meta_data)
    for row in svg_meta_data:
        svg_id = str(row[0])
        element_num = int(row[2])
        svg_file_name = svg_id + '.svg'
        nonempty_svg_data_list.append([svg_file_name, element_num])

In [10]:
for svg_file_attr_array in nonempty_svg_data_list:
    svg_file_name = svg_file_attr_array[0]
    element_num = svg_file_attr_array[1]
    # read the original svg object from simplified folder
    svg_file_path_name = os.path.join(svg_folder_path, svg_file_name)
    svg_obj = SVG.load_svg(svg_file_path_name)
    # transform svg object to tensor obect
    svg_tensor_content_obj = [svg_obj.to_tensor(concat_groups=False, PAD_VAL=-1)]
    svg_tensor_color_list = svg_obj.to_fillings()
    svg_tensor_obj = {"tensors": svg_tensor_content_obj, "fillings": svg_tensor_color_list}
    svg_tensor_file_name = svg_file_name.replace('.svg', '.pkl')
    svg_tensor_file_path_name = os.path.join(svg_tensor_folder_path, svg_tensor_file_name)
    # save tensor object to the file    
    save_svg_tensor(svg_tensor_file_path_name, svg_tensor_obj)

In [11]:
print(len(nonempty_svg_data_list))

560


Test the content of svg tensort object, it should contain two attributes, tensorts and fillings

In [13]:
svg_tensor_path = "dataset/svgs_tensor/104.pkl"
svg_tensor_obj = load_svg_tensor(svg_tensor_path)
print("svg_tensor_obj", svg_tensor_obj)

svg_tensor_obj {'tensors': [[tensor([[  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,   0.0000,
           0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 315.9119, 224.3979],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 315.9119,
         224.3979,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 150.5885, 287.7952],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 150.5885,
         287.7952,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 213.2533, 122.1927],
        [  6.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 213.2533,
         122.1927,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 315.9119, 224.3979]]), tensor([[  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,   0.0000,
           0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 405.0226, 206.0800],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 405.0226,
         206.0800,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 226.6363, 218.6486],
        [  1.00

In [14]:
icon_tensor_path = "dataset/icons_tensor/10.pkl"
icon_tensor_obj = load_svg_tensor(icon_tensor_path)
print("icon_tensor_obj", icon_tensor_obj)

icon_tensor_obj {'tensors': [[tensor([[  0.,  -1.,  -1.,  -1.,  -1.,  -1.,  39.,  39.,  -1.,  -1.,  -1.,  -1.,
          97.,  50.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1.,  97.,  50.,  -1.,  -1.,  -1.,  -1.,
          97.,  50.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1.,  97.,  50., 110.,  52., 119.,  56.,
         129.,  66.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1., 129.,  66.,  -1.,  -1.,  -1.,  -1.,
         129.,  66.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1., 129.,  66., 137.,  57., 149.,  53.,
         161.,  56.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1., 161.,  56., 170.,  59., 176.,  65.,
         179.,  73.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1., 179.,  73., 182.,  81., 182.,  90.,
         178.,  97.],
        [  2.,  -1.,  -1.,  -1.,  -1.,  -1., 178.,  97., 174., 104., 167., 109.,
         160., 113.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1., 160., 113.,  -1.,  -1.,  -1.,  -1.,
         150., 108.],
        [  1.,  -1.,  -1.,  -1.,  -1.,  -1.

# svgs_tensor process

In [15]:
def save_csv_file(csv_file_path_name, row_list):
    with open(csv_file_path_name,'rt') as file:
        writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        for row in row_list:
            writer.writerow(row)

In [16]:
svg_tensor_processed = []
svg_tensor_folder_path_processed = 'dataset/svgs_tensor_processed/'
svgs_meta_processed_path_name = 'dataset/svgs_meta_processed.csv'
with open('dataset/svgs_meta.csv','rt')as f:
    svg_meta_data = csv.reader(f)
    header = next(svg_meta_data)
    svg_tensor_processed.append(header)
    index = 0
    for row in svg_meta_data:
        svg_tensor_file_name = row[0] + '.pkl'
        svg_tensor_file_name_processed = str(index) + '.pkl'
        svg_tensor_file_path_name = os.path.join(svg_tensor_folder_path, svg_tensor_file_name)
        svg_tensor_file_path_name_processed = os.path.join(svg_tensor_folder_path_processed, svg_tensor_file_name_processed)
        # load svg tensor object
        svg_tensor_obj = load_svg_tensor(svg_tensor_file_path_name)
        # save svg tensor object
        save_svg_tensor(svg_tensor_file_path_name_processed, svg_tensor_obj)
        # reorder the first index 
        row[0] = str(index)
        svg_tensor_processed.append(row)
        index = index + 1

In [17]:
# save the processed svg tensor
pd.DataFrame(svg_tensor_processed).to_csv(svgs_meta_processed_path_name, header=None, index=None)

In [18]:
svg_tensor_path = "dataset/svgs_tensor_processed/10.pkl"
svg_tensor_obj = load_svg_tensor(svg_tensor_path)
print("svg_tensor_obj", svg_tensor_obj)

svg_tensor_obj {'tensors': [[tensor([[  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,   0.0000,
           0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 287.4915, 150.2848],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 287.4915,
         150.2848,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 224.8266, 315.8873],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 224.8266,
         315.8873,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 287.4915, 150.2848],
        [  6.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 287.4915,
         150.2848,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 287.4915, 150.2848]]), tensor([[  0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,   0.0000,
           0.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 405.2266, 209.4456],
        [  1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 405.2266,
         209.4456,  -1.0000,  -1.0000,  -1.0000,  -1.0000, 226.6421, 218.7860],
        [  1.00